# Calcul des rendements anormaux

In [2]:
#import 
from script import preprocess_isin, to_date # fonctions auxilliaires utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [3]:
# Déclaration du lieu de stockage des données initiales, et des fichiers de résultat
data_folder = "data/" 
result_folder = "results/"

**On va lire et traiter les tables de données correspondant aux dates d'annonce, aux cours boursiers, et aux indices de marché**

In [4]:
## lecture des données d'indices et de main equities
equities = preprocess_isin( path.join( data_folder, 'equities.csv' ), decimal = ',' , low_memory = False )
indices= pd.read_csv( path.join( data_folder, 'indices.csv' )).set_index('Date')

## Correspondance des dates entre les deux dataframe:
indices.drop(['10/10/2013'] , axis = 0, inplace = True)
equities.drop(['11/10/2023', '12/10/2023', '13/10/2023'], axis = 1, inplace = True)
equities.drop(equities.columns[0], axis = 1, inplace = True) #colonne inutile
equities.set_index('ISIN', inplace = True)

##dates d'annonces
dates = preprocess_isin('data/dates.csv', sep =';') # filtrage selon les ISIN
dates.set_index('ISIN', inplace = True)

## Création d'un regroupement par pays (indice du pays = 2 premières lettres de l'ISIN)
country = [ str[:2] for str in dates.index]
country_df = pd.DataFrame( {'Country' : country}, index = dates.index)
df = pd.concat([country_df, dates], axis = 1 )
groups = df.groupby('Country')


**On créer ensuite un dictionnaire pour faire la correspondance pays-indice boursier de référence**

In [5]:
print(indices.columns)


Index(['Unnamed: 0', 'S&P/ASX 200 ', 'Austrian Traded Index', 'BEL 20 Index',
       'S&P/TSX 60 INDEX', 'S&P/CLX IGPA (CLP) Index', 'Colombia Price Index',
       'OMXC 25 CAP Index', 'OMX Helsinki', 'IBEX 35 Index',
       'Budapest SE Index', 'Athex', 'OMX Iceland', 'ISEQ', 'FTSE', 'TOPIX ',
       'LuxX Index', 'OMX Vilnius', 'Oslo Stock Exchange Equity Index',
       'S&P/Bmv Ipc', ' Lisbon PSI Index', 'Amsterdam', 'Swiss Market Index',
       'OMX Stockholm', 'PX Prague', 'FTSE 100 Index', 'Shanghai SE 50',
       'BIST 100 Index', 'SBF 120 - PRICE INDEX',
       'DAX PERFORMANCE - PRICE INDEX', 'S&P 500 VALUE - PRICE INDEX'],
      dtype='object')


In [6]:
## Création d'un dictionnaire pour la correspondance pays-indice
#print(indices.columns)

dict = {'FR' : 'SBF 120 - PRICE INDEX'  , 'US' : 'S&P 500 VALUE - PRICE INDEX' , 
        'DE' : 'DAX PERFORMANCE - PRICE INDEX', 'AU' : 'S&P/ASX 200 ', 'AT' :'Austrian Traded Index',
        'BE' : 'BEL 20 Index', 'CA': 'S&P/TSX 60 INDEX', 'CL': 'S&P/CLX IGPA (CLP) Index' ,
        'CO': 'Colombia Price Index', 'DK': 'OMXC 25 CAP Index', 'ES': 'IBEX 35 Index', 
        'FI': 'OMX Helsinki', 'GR': 'Athex' , 'HU': 'Budapest SE Index', 'IE':'ISEQ' , 
        'IS': 'OMX Iceland', 'JP': 'TOPIX ' , 'LT': 'OMX Vilnius', 'LU' : 'LuxX Index', 'MX': 'S&P/Bmv Ipc', 
        'NO': 'Oslo Stock Exchange Equity Index' , 'NL': 'Amsterdam', 'PT': ' Lisbon PSI Index', 
        'SE': 'OMX Stockholm', 'CH': 'Swiss Market Index', 'GB': 'FTSE 100 Index', 'CZ': 'PX Prague', 
        'TR': 'BIST 100 Index' , 'CN': 'Shanghai SE 50' }

## Ce qui permet un accès rapide aux données pour un pays, exemple pour la France :
France = groups.get_group('FR') # contient les ISIN et les dates d'annonce correspondantes 
indice_fr = indices[dict['FR']] # contient le cours du CAC 40

### Calcul des rendements anormaux 

Les rendements anormaux considérés sont des rendements à 2 jours de marché après l'annonce, pour que l'effet de l'annonce soit intégré dans le cours boursier

In [7]:
# On définie une fonction qui calcule les rendements anormaux pour une entreprise i,
# avec t jours de données pour les régressions linéaires, à partir d'une dataframe country qui contient les
# entreprises d'un pays et leurs dates d'annonce, et du cours de l'indice boursier correspondant au pays

def anormal_return(country, indice_country, i, t):
    isin = country.index[i]
    date_annonce = to_date(dates.loc[isin])
    date = np.array(indice_country.index , dtype = str)

    # test de disponibilité du cours boursier
    try :
        equities.loc[isin]
    except :
        return ('cours non disponible', 'ERR')
    
    Y_company = equities.loc[isin][1:] 
    Y_indice = indice_country
    filter = np.logical_and( Y_company.notna(), Y_indice.notna() ) #dates où les 2 cours sont disponibles

    date = to_date(np.array(date[filter]))
    Y_company = np.array(Y_company[filter])
    Y_indice = np.array(Y_indice[filter])
    
    # test d'existence de données sur la période voulue
    try :  
       indice = np.where( date >= date_annonce)[0][0] #indice de l'annonce
    except :
        return ('pas de données sur la période voulue' , 'ERR')
      
    # calcul du rendement anormal
    y = Y_company[indice - t - 1 :indice] # t jours avant l'annonce
    x = Y_indice[indice - t - 1 :indice].reshape((-1,1))
    #instancier modèle
    model_linReg = LinearRegression()

    #test d'existence de données de régression
    try :
        model_linReg.fit(x, y)
    except :
        return ('pas de données pour les régressions' , 'ERR')
    
    #entrainement du modèle
    score = model_linReg.score(x,y)
    x_test = Y_indice[indice + 1].reshape((-1,1)) #  2 jours après l'annonce
    prediction = model_linReg.predict(x_test)[0]
    observation = Y_company[indice + 1]
    
    real = ( observation- Y_company[indice -1])/Y_company[indice -1] # rendement observé
    expected = ( prediction - Y_company[indice -1])/Y_company[indice -1] #rendement prédit
    ar = (real - expected)*100
    return (ar , score)

### Exemple d'utilistion :
anormal_return(France, indice_fr, 0, 3)

(-0.7048887183203169, 0.34079945495374375)

In [8]:
### on itère pour toutes les entrprises i d'un pays :

def Anormal_return(country, indice_country, t):
    AR =[]
    Coef = []
    for i in range(len(country.index)):
        ar = anormal_return(country, indice_country, i , t)[0]
        coef = anormal_return(country, indice_country, i , t)[1]
        AR.append(ar) 
        Coef.append(coef)
    return (AR , Coef)

### Exemple d'utilisation

(AR , Coef) = Anormal_return(France, indice_fr, 3)
print(AR)
print(Coef)

[-0.7048887183203169, 1.3655406126071228, -0.4474202833527879, -5.9206364478497395, 'pas de données sur la période voulue', -6.7796045533422316, 5.488191778219426, 3.6153590217149674, -1.3080547529160562, -0.3263952099188894, 8.831101348638223, -2.6340728311016304, 0.5061983483106013, 'cours non disponible', 'cours non disponible', -1.9579965585557073, 3.523178133854657, 1.716555294089373, 4.50219275358173, 1.3621877243815916, 5.420235987553853, 1.2808056424065255, -2.3283471547013863, 1.2123609428518498, -0.15978954361756323, -0.1442801764223069, 0.20323789812725715, -0.9445141597548707, 3.4658608315600747, 0.2938919630684679, -0.2985881935338523, 0.1992144076594926, -3.2018829034709704, 'pas de données sur la période voulue', 0.7607843705332998, 0.6019713015208439, 'cours non disponible', -1.005510175735147, -11.704685015926723, -2.7206673771235494, 1.2531482141556256, 'cours non disponible', 4.548159674180763, 2.1892045538834544, 0.547052740856426, -1.2257832209370165, 1.62036985993

In [9]:
# On veut maintenant obtenir une dataframe contenant les rendements anormaux pour un pays et les scores 
# de regréssion, calculé avec différentes fenêtres temporelles ( ici de 4 à 20 jours, par pas de 2 jours) 

def Result_DataFrame( country, indice ):
    isin = np.array(country.index)
    df_resultat = pd.DataFrame( {'ISIN' : isin})
    df_resultat.set_index('ISIN', inplace = True)
    # Calcul des différents rendements anormaux
    for i in range(2,11):
       ar  = Anormal_return(country, indice, 2*i)[0]
       coef = Anormal_return(country, indice, 2*i)[1]
       df_temporaire = pd.DataFrame({ 'ISIN' : isin , f"AR_{2*i}" : ar, f"Coef_{2*i}" : coef})
       df_resultat = df_resultat.join( df_temporaire.set_index('ISIN'))
    return df_resultat

# Exemple d'utilisation (Allemagne ici)
Country = groups.get_group('DE')
Indice = indices[dict['DE']]

df_resultat = Result_DataFrame(Country, Indice)
#print( df_resultat)  

In [10]:
# on peut maintenant obtenir une dataframe pour chaque pays. 
# On va enfin concaténer toutes les dataframes pour obtenir une grosse base de données internationale sur laquelle
# on pourra procéder à une étude d'hétérogénéité :
# Très long à exécuter,  les résultats ont déjà été sauvegardé dans le CSV 'AR.csv'
result = pd.DataFrame()

for country in dict :
    Country = groups.get_group(country)
    Indice = indices[dict[country]]
    df_temporaire = Result_DataFrame( Country, Indice )
    result = pd.concat([result, df_temporaire])

print(result)


                                              AR_4    Coef_4  \
ISIN                                                           
FR0000120404                             -0.588014  0.187373   
FR0000120073                              1.517742  0.489255   
FR0010220475                              0.345573   0.05387   
FR0000051732                             -5.942007  0.598023   
FR0000121857  pas de données sur la période voulue       ERR   
...                                            ...       ...   
TRAARCLK91H5                               0.86596  0.180638   
TRABRISA91E3                             -4.995113  0.924314   
TRAVESTL91H6                             -3.199047  0.003472   
CNE0000019B0                              -0.08279  0.285058   
CNE100000NC4                             -7.753864  0.029276   

                                              AR_6    Coef_6  \
ISIN                                                           
FR0000120404                           

In [11]:
# Pour sauvegarder les résultats :
result.to_csv( path.join( result_folder , 'AR.csv'))